In [ ]:
# Step 1: Load core ICU info
import pandas as pd
import numpy as np

# Load ICU stays
icustays = pd.read_csv("icustays.csv.gz", compression="gzip", parse_dates=["intime", "outtime"])
icu_meta = icustays[["subject_id", "hadm_id", "stay_id", "intime", "first_careunit", "los"]]

# Load hospital admission info and assign mortality label
admissions = pd.read_csv("admissions.csv.gz", compression='gzip')
df_mortality = admissions.merge(icustays, on=["subject_id", "hadm_id"])
df_mortality["mortality_label"] = df_mortality["hospital_expire_flag"]
df_mortality = df_mortality[["subject_id", "hadm_id", "stay_id", "mortality_label"]]

# Step 2: Generate structured_note from icustays
icu_meta["structured_note"] = icu_meta.apply(
    lambda row: f"The patient was admitted to the {row['first_careunit']} and stayed for {round(row['los'], 1)} days.",
    axis=1
)

# Step 3: Load procedure events
procedures = pd.read_csv("procedureevents.csv.gz", compression="gzip", parse_dates=["starttime"])

# Step 4: Load d_items to decode procedure names
items = pd.read_csv("d_items.csv.gz", compression="gzip")
proc_map = items.set_index("itemid")["label"].to_dict()
procedures["proc_name"] = procedures["itemid"].map(proc_map)

# Step 5: Merge intime into procedures
proc_with_time = procedures.merge(icustays[["subject_id", "stay_id", "intime"]], on=["subject_id", "stay_id"], how="left")

# Keep only those within 24h of ICU admission
proc_with_time = proc_with_time[
    (proc_with_time["starttime"] >= proc_with_time["intime"])
    & (proc_with_time["starttime"] <= proc_with_time["intime"] + pd.Timedelta(hours=24))
]

# Step 6: Aggregate procedure descriptions
proc_summaries = proc_with_time.groupby(["subject_id", "hadm_id", "stay_id"])['proc_name'].apply(lambda x: list(set(x.dropna()))).reset_index()

# Turn procedure list into sentence
proc_summaries["procedure_note"] = proc_summaries["proc_name"].apply(
    lambda x: "The patient received the following procedures within the first 24 hours: " + ", ".join(x) + "." if x else "",
)

# Step 7: Merge all back together
icu_enriched = icu_meta.merge(proc_summaries[["subject_id", "hadm_id", "stay_id", "procedure_note"]],
                              on=["subject_id", "hadm_id", "stay_id"], how="left")

icu_enriched["procedure_note"] = icu_enriched["procedure_note"].fillna("")

# Step 8: Add discharge notes
print("Loading discharge notes...")
discharge = pd.read_csv("discharge.csv.gz", compression='gzip')
discharge_latest = discharge.sort_values("charttime").drop_duplicates("hadm_id", keep="last")
discharge_latest["text_note"] = discharge_latest["text"].str.replace(r"\[\*\*.*?\*\*\]", "", regex=True)
df_mortality = df_mortality.merge(discharge_latest[["hadm_id", "text_note"]], on="hadm_id", how="inner")

# Step 9: Add lab-based vitals from labevents
print("Loading lab events for vitals...")
labs = pd.read_csv("labevents.csv.gz", compression="gzip", parse_dates=["charttime"])

# Example lab-based vital signs and proxies
vital_lab_items = {
    50912: "glucose",
    50983: "sodium",
    50822: "potassium",
    51006: "bun",
    50971: "creatinine",
    50868: "calcium",
    50862: "chloride",
    50902: "magnesium",
    50809: "albumin",
    50820: "bilirubin_total"
}

labs = labs[labs["itemid"].isin(vital_lab_items.keys())]
labs["vital_label"] = labs["itemid"].map(vital_lab_items)
# Clean value column to keep only numeric values
labs["value"] = pd.to_numeric(labs["value"], errors="coerce")
labs = labs.dropna(subset=["value"])

# Now safely aggregate
lab_summary = labs.groupby(["subject_id", "hadm_id", "itemid"]).agg({"value": "median"}).reset_index()

lab_summary["vital_label"] = lab_summary["itemid"].map(vital_lab_items)

lab_pivot = lab_summary.pivot_table(index=["subject_id", "hadm_id"], columns="vital_label", values="value").reset_index()
lab_pivot.columns.name = None

# Step 10: Add CXR image path
print("Loading CXR metadata...")
study_df = pd.read_csv("cxr-study-list.csv.gz", compression="gzip")
record_df = pd.read_csv("cxr-record-list.csv.gz", compression="gzip")
cxr_meta = study_df.merge(record_df, on=["subject_id", "study_id"], how="inner")
cxr_meta["image_path"] = cxr_meta["dicom_id"].apply(
    lambda x: f"mimic-cxr/files/p{str(x)[:2]}/p{str(x)}/s{x}/{x}.jpg.gz"
)
cxr_meta_dedup = cxr_meta.sort_values("study_id").drop_duplicates("subject_id", keep="first")

# Step 11: Merge everything into one final dataset
df_full = df_mortality.merge(icu_enriched, on=["subject_id", "hadm_id", "stay_id"], how="left")
df_full = df_full.merge(lab_pivot, on=["subject_id", "hadm_id"], how="left")
df_full = df_full.merge(cxr_meta_dedup[["subject_id", "image_path"]], on="subject_id", how="inner")
df_full["combined_note"] = df_full["procedure_note"].fillna("") + " " + df_full["structured_note"].fillna("") + " " + df_full["text_note"].fillna("")

# Save final result
df_full.to_csv("final_multimodal_dataset.csv", index=False)

# Output summary
print("\nFinal dataset columns:")
print(list(df_full.columns))


In [ ]:
# Drop any column with too much missing value
clean_dataset = filtered_dataset.drop(columns=['potassium', 'albumin', 'chloride', 'bilirubin_total'])

# Drop all rows with any missing values
clean_dataset = clean_dataset.dropna()

# Optional: check new size
print(f"Remaining rows after dropping missing values: {len(clean_dataset)}")


## add image information

In [ ]:
cxr_records = pd.read_csv("data_process/cxr-record-list.csv.gz", compression="gzip")

In [ ]:
# Deduplicate to get one dicom_id + study_id per subject
cxr_records_dedup = cxr_records.sort_values("study_id").drop_duplicates("subject_id", keep="first")

# Merge to bring in study_id and dicom_id
df_merged = clean_dataset.merge(cxr_records_dedup[["subject_id", "study_id", "dicom_id"]], on="subject_id", how="inner")
df_merged["image_path"] = df_merged.apply(
    lambda row: f"files/p{str(row['subject_id']).zfill(8)[:2]}/p{str(row['subject_id']).zfill(8)}/s{int(row['study_id'])}/{row['dicom_id']}.dcm",
    axis=1
)

In [ ]:
df_merged
print(df_merged.columns.tolist())

In [ ]:
df_merged.to_csv("final_image.csv", index=False)

In [ ]:
final = df_merged.drop(columns=['study_id', 'dicom_id', 'subject_id', 'hadm_id', 'stay_id'])

In [ ]:
final.to_csv("final.csv", index=False)
